In [ ]:
#%pip install pandas-datareader

In [16]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
from keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import layers
from tickerData import *
from tqdm import tqdm_notebook, tqdm
from collections import deque

In [3]:
df = getTickerData(ticker="btc-usd", period="max", interval="1d")
df.drop(columns={'Adj Close'}, inplace=True)
df.rename(columns={'Date':'date', 'Open':'open', 'High':'high','Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)
df

[*********************100%***********************]  1 of 1 completed


,date,open,high,low,close,volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100
...,...,...,...,...,...,...
3100,2023-03-14,24201.765625,26514.716797,24081.183594,24746.074219,54622230164
3101,2023-03-15,24770.925781,25240.615234,23964.910156,24375.960938,43655701450
3102,2023-03-16,24373.457031,25190.326172,24225.111328,25052.789062,33866061747
3103,2023-03-17,25055.123047,27787.812500,24955.169922,27423.929688,50730261335


In [27]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def state_creator(data, timestep, window_size):
  
    starting_id = timestep - window_size + 1
  
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = starting_id * [data[0]] + list(data[0:timestep+1])
    
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
    return np.array([state])

def stock_price_format(n):
    if n < 0:
        return "- # {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [52]:
class AI_Trader():

    def model_builder(self):
    
        model = tf.keras.models.Sequential()     
        model.add(Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

        return model
    
    def __init__(self, state_size, action_space=3, model_name="AITrader"):
        
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(range(1,2000))
        self.inventory = []
        self.model_name = model_name
    
        self.gammsa = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        self.model = self.model_builder()
    
    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        actions = self.model.predict(actions[0])

    def batch_train(self, batch_size):
        
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target = self.model.predict(state)
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay
        
    def dataset_loader(df):

        dataset = df
  
        start_date = str(dataset.index[0]).split()[0]
        end_date = str(dataset.index[1]).split()[0]
  
        data = dataset['close']
  
        return data
      

In [53]:
data = AI_Trader.dataset_loader(df)
data

0         457.334015
1         424.440002
2         394.795990
3         408.903992
4         398.821014
            ...     
3100    24746.074219
3101    24375.960938
3102    25052.789062
3103    27423.929688
3104    27456.119141
Name: close, Length: 3105, dtype: float64

In [50]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [23]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 32)                352       
                                                                 
 dense_13 (Dense)            (None, 64)                2112      
                                                                 
 dense_14 (Dense)            (None, 128)               8320      
                                                                 
 dense_15 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [55]:
for episode in range(1, episodes + 1):
  
    print("Episode: {}/{}".format(episode, episodes))
  
    state = state_creator(data, 0, window_size + 1)
  
    total_profit = 0
    trader.inventory = []
  
    for t in tqdm(range(data_samples)):
    
        action = trader.trade(state)
    
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0
    
        if action == 1: #Buying
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stock_price_format(data[t]))
      
        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)
      
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )
      
        if t == data_samples - 1:
            done = True
        else:
            done = False
      
        trader.memory.append((state, action, reward, next_state, done))
    
        state = next_state
    
        if done:
            print("########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("########################")
    
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
      
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))

Episode: 1/1000


IndexError: list index out of range